<a href="https://colab.research.google.com/github/Fackor/Advanced_Machine_Learning_Course/blob/main/Zero_Shot_Pipeline_for_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==3.1.0

     |████████████████████████████████| 890kB 8.6MB/s 
     |████████████████████████████████| 3.0MB 18.3MB/s 
     |████████████████████████████████| 1.1MB 47.8MB/s 
     |████████████████████████████████| 890kB 54.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b96c9b79206940c13e2a837e97481b25dce81203813abcaa64020a3d573de0f5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import pipeline

In [3]:
#classifier = pipeline("zero-shot-classification")
classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def zeroshot_NER(paragraph, candidate_labels, threshold):
  words = [word.lower() for word in paragraph.split() if word.isalpha()]
  NER = []
  for word in words:
    output = classifier(word, candidate_labels)

    label =  output['labels'][0] if output['scores'][0] > threshold else '[UNK]'
    NER.append([word, label])

  return NER  

In [8]:
sequence = input("Enter a paragraph to do NER on.")
candidate_labels = input("Enter candidate labels separated by double colon(::).").split("::")
candidate_labels.append('[UNK]')

NER = zeroshot_NER(sequence, candidate_labels, 0.4)
for each in NER:
  print(f"{each[0]}<{each[1]}>", end=" ")

Enter a paragraph to do NER on.Kolkata (/kɒlˈkɑːtə/[15] or /kɒlˈkʌtə/,[16] Bengali: [ˈkolˌkata] (About this soundlisten), also rendered Calcutta /kælˈkʌtə/,[16] the official name until 2001) is the capital of the Indian state of West Bengal. Located on the eastern bank of the Hooghly River, the city is approximately 80 kilometres (50 mi) west of the border with Bangladesh. It is the prime business, commercial, and financial hub of eastern India and the main port of communication for the North-East Indian states,[17] as well as having the third-largest urban economy of India.[13] According to the 2011 Indian census, Kolkata is the seventh-most populous city in India, with a population of 4.5 million residents within the city limits, and a population of over 14.1 million residents in the Kolkata Metropolitan Area, making it the third-most populous metropolitan area in India. The Port of Kolkata is India's oldest operating port and its sole major riverine port. Kolkata is known as the "cu

['who', 'NULL']
['are', 'NULL']
['you', 'NULL']
['voting', 'politics']
['for', 'NULL']
['in', 'NULL']
